In [60]:
import os
import subprocess
import time
import sys
import threading
import socket
from urllib.request import Request, urlopen
import re
!pip install streamlit pyjwt bcrypt python-dotenv pyngrok -q
!pip install streamlit pyjwt bcrypt python-dotenv pyngrok nltk streamlit-option-menu plotly textstat PyPDF2 -q

!pip install bcrypt

# Function to install packages
def install_packages():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "streamlit", "pyngrok", "pyjwt", "watchdog"])
print("Installing required packages...")
install_packages()

# Import after installation
from pyngrok import ngrok

# --- Create Streamlit Config for Dark Theme ---
os.makedirs(".streamlit", exist_ok=True)
config_toml = """
[theme]
base="dark"
primaryColor="#00F5FF"
backgroundColor="#041C32"
secondaryBackgroundColor="#06283D"
textColor="#FFFFFF"
font="sans serif"

[server]
headless = true
"""
with open(".streamlit/config.toml", "w") as f:
    f.write(config_toml)

print("Applied Neon Dark Theme configuration.")

Installing required packages...
Applied Neon Dark Theme configuration.


In [61]:
%%writefile db.py
# ---------------- DATABASE ----------------
import sqlite3
import bcrypt
import datetime
import time

DB_NAME = "users.db"
max_login_attempts = 3
lockout_time = 300  # 5 minutes in seconds


def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    # Users table
    c.execute('''
    CREATE TABLE IF NOT EXISTS users (
        email TEXT PRIMARY KEY,
        username TEXT,
        password BLOB,
        security_question TEXT,
        security_answer BLOB,
        created_at TEXT
    )
    ''')

    # Password History table
    c.execute('''
    CREATE TABLE IF NOT EXISTS password_history (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        email TEXT,
        password BLOB,
        set_at TEXT,
        FOREIGN KEY(email) REFERENCES users(email)
    )
    ''')

    # Login Attempts table
    c.execute('''
    CREATE TABLE IF NOT EXISTS login_attempts (
        email TEXT PRIMARY KEY,
        attempts INTEGER,
        last_attempt REAL
    )
    ''')

    conn.commit()
    conn.close()

    # Ensure Admin Exists
    init_admin()

def get_username(email):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT username FROM users WHERE email=?", (email,))
    row = cursor.fetchone()
    return row[0] if row else None

def init_admin():
    if not check_user_exists("admin@llm.com"):
        register_user("Admin", "admin@llm.com", "Admin@123Secure!", "Admin default question", "admin")
        print("Admin account created.")

def _get_timestamp():
    return datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

# --- Rate Limiting ---
def get_login_attempts(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT attempts, last_attempt FROM login_attempts WHERE email = ?", (email,))
    data = c.fetchone()
    conn.close()
    return data if data else (0, 0)

def increment_login_attempts(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    attempts, _ = get_login_attempts(email)
    new_attempts = attempts + 1
    now = time.time()

    c.execute("INSERT OR REPLACE INTO login_attempts (email, attempts, last_attempt) VALUES (?, ?, ?)",
              (email, new_attempts, now))
    conn.commit()
    conn.close()

def reset_login_attempts(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("DELETE FROM login_attempts WHERE email = ?", (email,))
    conn.commit()
    conn.close()

def is_rate_limited(email):
    attempts, last_attempt = get_login_attempts(email)
    if attempts >= max_login_attempts:
        if time.time() - last_attempt < lockout_time:
            return True, lockout_time - (time.time() - last_attempt)
        else:
            # Lockout expired
            reset_login_attempts(email)
    return False, 0

# --- User Management ---
def register_user(username, email, password, question, answer):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    try:
        hashed_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
        hashed_answer = bcrypt.hashpw(answer.encode(), bcrypt.gensalt())
        now = _get_timestamp()

        c.execute("""
        INSERT INTO users (email, username, password, security_question, security_answer, created_at)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (email, username, hashed_password, question, hashed_answer, now))

        c.execute("""
        INSERT INTO password_history (email, password, set_at)
        VALUES (?, ?, ?)
        """, (email, hashed_password, now))

        conn.commit()
        return True

    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()
def get_security_question(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT security_question FROM users WHERE email = ?", (email,))
    data = c.fetchone()
    conn.close()
    return data[0] if data else None


def verify_security_answer(email, answer):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT security_answer FROM users WHERE email = ?", (email,))
    data = c.fetchone()
    conn.close()

    if data:
        return bcrypt.checkpw(answer.encode(), data[0])

    return False
def get_security_question(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT security_question FROM users WHERE email = ?", (email,))
    data = c.fetchone()
    conn.close()
    return data[0] if data else None


def verify_security_answer(email, answer):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT security_answer FROM users WHERE email = ?", (email,))
    data = c.fetchone()
    conn.close()

    if data:
        stored_hash = data[0]
        return bcrypt.checkpw(answer.encode('utf-8'), stored_hash)

    return False

def authenticate_user(email, password):
    # Check rate limit first
    locked, _ = is_rate_limited(email)
    if locked:
        return "locked"

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password FROM users WHERE email = ?", (email,))
    data = c.fetchone()
    conn.close()

    if data:
        stored_hash = data[0]
        if bcrypt.checkpw(password.encode('utf-8'), stored_hash):
            reset_login_attempts(email)
            return True

    increment_login_attempts(email)
    return False

def check_is_old_password(email, password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password, set_at FROM password_history WHERE email = ? ORDER BY set_at DESC", (email,))
    history = c.fetchall()
    conn.close()

    for stored_hash, set_at in history:
        if bcrypt.checkpw(password.encode('utf-8'), stored_hash):
            return set_at
    return None

def check_password_reused(email, new_password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password FROM password_history WHERE email = ?", (email,))
    history = c.fetchall()
    conn.close()

    for (stored_hash,) in history:
        if bcrypt.checkpw(new_password.encode('utf-8'), stored_hash):
            return True
    return False

def check_user_exists(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT 1 FROM users WHERE email = ?", (email,))
    data = c.fetchone()
    conn.close()
    return data is not None

def update_password(email, new_password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()

    salt = bcrypt.gensalt()
    hashed = bcrypt.hashpw(new_password.encode('utf-8'), salt)
    now = _get_timestamp()

    c.execute("UPDATE users SET password = ? WHERE email = ?", (hashed, email))
    c.execute("INSERT INTO password_history (email, password, set_at) VALUES (?, ?, ?)", (email, hashed, now))

    conn.commit()
    conn.close()

# --- Admin Functions ---
def get_all_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT email, created_at FROM users")
    data = c.fetchall()
    conn.close()
    return data

def delete_user(email):
    if email == "admin@llm.com": return False # Protect admin
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("DELETE FROM users WHERE email = ?", (email,))
    c.execute("DELETE FROM password_history WHERE email = ?", (email,))
    c.execute("DELETE FROM login_attempts WHERE email = ?", (email,))
    conn.commit()
    conn.close()
    return True
    conn.commit()

Overwriting db.py


In [62]:
%%writefile readability.py
import textstat

class ReadabilityAnalyzer:
    def __init__(self, text):
        self.text = text
        self.num_sentences = textstat.sentence_count(text)
        self.num_words = textstat.lexicon_count(text, removepunct=True)
        self.num_syllables = textstat.syllable_count(text)
        self.complex_words = textstat.difficult_words(text)
        self.char_count = textstat.char_count(text)

    def get_all_metrics(self):
        return {
            "Flesch Reading Ease": textstat.flesch_reading_ease(self.text),
            "Flesch-Kincaid Grade": textstat.flesch_kincaid_grade(self.text),
            "SMOG Index": textstat.smog_index(self.text),
            "Gunning Fog": textstat.gunning_fog(self.text),
            "Coleman-Liau": textstat.coleman_liau_index(self.text)
        }

Overwriting readability.py


In [63]:
%%writefile app.py
import streamlit as st
import jwt
import datetime
import sqlite3
import re
import os
import bcrypt
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import secrets
import time
import hmac
import hashlib
import struct
import db
from streamlit_option_menu import option_menu
import plotly.graph_objects as go
import PyPDF2

st.markdown("""
<style>
.stApp {
    background: linear-gradient(to right, #031926, #062f45);
    color: white;
}
</style>
""", unsafe_allow_html=True)

# ---------------- CONFIG ----------------
SECRET_KEY = "super_secret_key"
ALGORITHM = "HS256"

st.set_page_config(
    page_title="Infosys Springboard",
    page_icon="🎓",
    layout="centered"
)
# Configuration
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
SECRET_KEY = os.getenv("JWT_SECRET", "super-secret-key-change-this")
EMAIL_ADDRESS = "madhavibenparmar22@gnu.ac.in"
OTP_EXPIRY_MINUTES = 10

# Initialize DB
if 'db_initialized' not in st.session_state:
    db.init_db()
    st.session_state['db_initialized'] = True


# ---------------- NEON CSS ----------------
st.markdown("""
<style>

/* Main Background */
.stApp {
    background: linear-gradient(135deg, #041C32, #06283D);
}

/* Glass Card */
.neon-card {
    background: rgba(255,255,255,0.05);
    backdrop-filter: blur(15px);
    padding: 40px;
    border-radius: 20px;
    border: 1px solid rgba(0,245,255,0.5);
    box-shadow: 0 0 25px rgba(0,245,255,0.4);
    animation: fadeIn 0.8s ease-in-out;
}

/* Titles */
h1, h2, h3 {
    color: #00F5FF;
    text-align: center;
}

/* Inputs */
.stTextInput>div>div>input {
    background: transparent;
    border: none;
    border-bottom: 2px solid #00F5FF;
    color: white;
    font-size: 16px;
}

.stSelectbox>div>div {
    background: rgba(255,255,255,0.05);
    border: 1px solid #00F5FF;
    border-radius: 10px;
    color: white;
}

/* Buttons */
.stButton>button {
    width: 100%;
    border-radius: 30px;
    padding: 12px;
    font-weight: bold;
    border: none;
    background: linear-gradient(90deg, #00F5FF, #00C9A7);
    color: black;
    transition: 0.3s ease;
    box-shadow: 0 0 15px rgba(0,245,255,0.6);
    white-space: nowrap !important;
}

.stButton>button:hover {
    transform: scale(1.05);
    box-shadow: 0 0 25px rgba(0,245,255,1);
}

/* Sidebar */
[data-testid="stSidebar"] {
    background: #031926;
}

/* Animation */
@keyframes fadeIn {
    from {opacity: 0; transform: translateY(20px);}
    to {opacity: 1; transform: translateY(0);}
}

</style>
""", unsafe_allow_html=True)


# ---------------- JWT ----------------
def create_token(data):
    data["exp"] = datetime.datetime.utcnow() + datetime.timedelta(hours=1)
    return jwt.encode(data, SECRET_KEY, algorithm=ALGORITHM)

# ---------------- VALIDATION ----------------
def get_relative_time(date_str):
    if not date_str: return "some time ago"
    try:
        past = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        diff = datetime.datetime.utcnow() - past
        days = diff.days
        if days > 365: return f"{days // 365} years ago"
        elif days > 30: return f"{days // 30} months ago"
        elif days > 0: return f"{days} days ago"
        else: return "recently"
    except: return date_str
def valid_email(email):
    pattern = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'
    return re.match(pattern, email)

def valid_password(password):
    pattern = r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[@$!%*?&])[A-Za-z\d@$!%*?&]{8,}$'
    return re.match(pattern, password)

def password_strength(password):
    score = 0
    if len(password) >= 8: score += 1
    if re.search(r'[A-Z]', password): score += 1
    if re.search(r'[a-z]', password): score += 1
    if re.search(r'\\d', password): score += 1
    if re.search(r'[@$!%*?&]', password): score += 1
    return score
# --- Security Logic ---

def generate_otp():
    secret = secrets.token_bytes(20)
    counter = int(time.time())
    msg = struct.pack(">Q", counter)
    hmac_hash = hmac.new(secret, msg, hashlib.sha1).digest()
    offset = hmac_hash[19] & 0xf
    code = ((hmac_hash[offset] & 0x7f) << 24 | (hmac_hash[offset + 1] & 0xff) << 16 | (hmac_hash[offset + 2] & 0xff) << 8 | (hmac_hash[offset + 3] & 0xff))
    otp = code % 1000000
    return f"{otp:06d}"

def create_otp_token(otp, email):
    otp_hash = bcrypt.hashpw(otp.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')
    payload = {
        'otp_hash': otp_hash,
        'sub': email,
        'type': 'password_reset',
        'iat': datetime.datetime.utcnow(),
        'exp': datetime.datetime.utcnow() + datetime.timedelta(minutes=OTP_EXPIRY_MINUTES)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm='HS256')

def verify_otp_token(token, input_otp, email):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=['HS256'])
        if payload.get('sub') != email: return False, "Token mismatch"
        if bcrypt.checkpw(input_otp.encode('utf-8'), payload['otp_hash'].encode('utf-8')):
            return True, "Valid"
        return False, "Invalid OTP"
    except Exception as e:
        return False, str(e)

def send_email(to_email, otp, app_pass):
    msg = MIMEMultipart()
    msg['From'] = f"Infosys Springboard <{EMAIL_ADDRESS}>"
    msg['To'] = to_email
    msg['Subject'] = "🔐 Infosys Springboard - Password Reset OTP"

    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            .container {{
                font-family: Arial, sans-serif;
                background-color: #0e1117;
                padding: 40px;
                text-align: center;
                color: #ffffff;
            }}
            .card {{
                background-color: #1f2937;
                border-radius: 12px;
                padding: 30px;
                max-width: 500px;
                margin: auto;
                border: 1px solid #00F5FF;
            }}
            .otp-box {{
                font-size: 32px;
                font-weight: bold;
                letter-spacing: 6px;
                padding: 20px;
                margin: 20px 0;
                background: #00F5FF;
                color: black;
                border-radius: 8px;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="card">
                <h2>Infosys Springboard Security</h2>
                <p>Use this OTP to reset your password for {to_email}</p>
                <div class="otp-box">{otp}</div>
                <p>Valid for {OTP_EXPIRY_MINUTES} minutes</p>
            </div>
        </div>
    </body>
    </html>
    """

    msg.attach(MIMEText(body, 'html'))

    try:
        s = smtplib.SMTP('smtp.gmail.com', 587)
        s.starttls()
        s.login(EMAIL_ADDRESS, app_pass if app_pass else EMAIL_PASSWORD)
        s.sendmail(EMAIL_ADDRESS, to_email, msg.as_string())
        s.quit()
        return True, "Sent"
    except Exception as e:
        return False, str(e)

# ---------------- SESSION ----------------
if "page" not in st.session_state:
    st.session_state.page = "login"

if "token" not in st.session_state:
    st.session_state.token = None

def switch_page(page):
    st.session_state['page'] = page
    st.rerun()

def logout():
    st.session_state['user'] = None
    st.session_state['page'] = 'login'
    st.rerun()

# ---------------- SIGNUP ----------------
def signup():
    st.markdown("""
    <div class="neon-card" style="
        display:flex;
        justify-content:center;
        align-items:center;
        text-align:center;
        font-size:28px;
        font-weight:600;
        letter-spacing:0.5px;
        color:#ffffff;
        text-shadow:0 0 12px rgba(0,245,255,0.8);
    ">
        Welcome to Infosys Springboard Internship
    </div>
    """, unsafe_allow_html=True)
    st.title("Create Account")

    questions = [
        "What is your pet name?",
        "What is your mother’s maiden name?",
        "What is your favorite teacher?",
        "What was your first school name?",
        "What is your favorite food?"
    ]

    with st.form("signup_form"):
        username = st.text_input("Username")
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        confirm = st.text_input("Confirm Password", type="password")
        question = st.selectbox("Security Question", questions)
        answer = st.text_input("Security Answer")
        submit = st.form_submit_button("Signup")

        # Show password strength
        if password:
            score = password_strength(password)
            st.text(f"Password strength: {score}/5")

        if submit:
            username = username.strip()
            email = email.strip()
            password = password.strip()
            confirm = confirm.strip()
            answer = answer.strip()

            if not username:
                st.error("Username cannot be empty or spaces")
            elif not email:
                st.error("Email cannot be empty or spaces")
            elif not password:
                st.error("Password cannot be empty or spaces")
            elif not confirm:
                st.error("Confirm Password cannot be empty")
            elif not answer:
                st.error("Security Answer cannot be empty or only spaces")
            elif not valid_email(email):
                st.error("Invalid Email format")
            elif not valid_password(password):
                st.error("Password must be strong (8+ chars, uppercase, lowercase, number, special char)")
            elif password != confirm:
                st.error("Passwords do not match")
            elif not all([username,email,password,confirm,answer]):
                st.error("All fields are mandatory")
            elif not valid_email(email):
                st.error("Invalid Email format")
            elif not valid_password(password):
                st.error("Password must be strong (8+ chars, uppercase, lowercase, number, special char)")
            elif password != confirm:
                st.error("Passwords do not match")
            else:
                existing_user = db.check_user_exists(email)

                if existing_user:
                    st.error("Email already registered")
                elif db.register_user(username, email, password, question, answer):
                    st.success("Account created successfully 🎉")
                else:
                    st.error("User already exists.")

    st.markdown("<br><br>", unsafe_allow_html=True)

    if st.button("Go to Login"):
        st.session_state.page="login"
        st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)




# ---------------- LOGIN ----------------
def login():
    st.markdown("""
    <div class="neon-card" style="
        display:flex;
        justify-content:center;
        align-items:center;
        text-align:center;
        font-size:28px;
        font-weight:600;
        letter-spacing:0.5px;
        color:#ffffff;
        text-shadow:0 0 12px rgba(0,245,255,0.8);
    ">
        Welcome to Infosys Springboard Internship
    </div>
    """, unsafe_allow_html=True)
    st.title("Login")

    with st.form("login_form"):
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        submit = st.form_submit_button("Login")

        if submit:
            auth = db.authenticate_user(email, password)

            if auth == "locked":
                st.error("Account locked. Try again later.")

            elif auth:
                if email == "admin@llm.com":
                    username = "Admin"
                else:
                    username = db.get_username(email)

                st.session_state['user'] = username if username else email

                st.session_state['user'] = email
                st.session_state.token = create_token({
                    "email": email,
                    "username": username
                })
                st.success("Login Successful! 🎉")
                time.sleep(1)

                # Admin check
                if email == "admin@llm.com":
                    st.session_state.page = "admin_dashboard"
                else:
                    st.session_state.page = "dashboard"

                st.rerun()

            else:
                # Invalid login
                st.error("Invalid credentials.")

                # Check if old password used
                old_dt = db.check_is_old_password(email, password)
                if old_dt:
                    st.warning(f"Note: You used an old password from {get_relative_time(old_dt)}.")

    col1, space, col2 = st.columns([1, 2, 1])
    with col1:
        if st.button("Forgot Password",use_container_width=True):
            st.session_state.page="forgot"
            st.rerun()
    with col2:
        if st.button("Create Account",use_container_width=True):
            st.session_state.page="signup"
            st.rerun()

    st.markdown('</div>', unsafe_allow_html=True)

# ---------------- DASHBOARD ----------------
def dashboard():

    st.markdown("""
    <div class="neon-card" style="
        display:flex;
        justify-content:center;
        align-items:center;
        text-align:center;
        font-size:28px;
        font-weight:600;
        letter-spacing:0.5px;
        color:#ffffff;
        text-shadow:0 0 12px rgba(0,245,255,0.8);
    ">
        Welcome to Infosys Springboard Internship
    </div>
    """, unsafe_allow_html=True)
    if not st.session_state['user']: switch_page('login'); return
    st.markdown("<br><br>", unsafe_allow_html=True)

    st.markdown(
    f"<div style='font-size:31px; text-align:center; font-weight:bold;'>🤖 Welcome, {username}</div>",
    unsafe_allow_html=True
    )

    st.markdown(
        "<div style='font-size:29px; text-align:center; font-weight:bold;'>🤖 Infosys Springboard Chat</div>",
        unsafe_allow_html=True
    )
    if "messages" not in st.session_state:
        st.session_state.messages = []

    for msg in st.session_state.messages:
        with st.chat_message(msg["role"]):
            st.markdown(msg["content"])

    if prompt := st.chat_input("Ask me anything..."):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            response = f"Simulated Response to: {prompt} (This is a Secure Mock)"
            st.markdown(response)
            st.session_state.messages.append({"role": "assistant", "content": response})

    st.markdown('</div>', unsafe_allow_html=True)

def create_gauge(value, title, min_val, max_val, color):
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        title={"text": title},
        gauge={
            "axis": {"range": [min_val, max_val]},
            "bar": {"color": color},
            "steps": [
                {"range": [min_val, (min_val+max_val)/3], "color": "#1f2937"},
                {"range": [(min_val+max_val)/3, (min_val+max_val)*2/3], "color": "#374151"},
                {"range": [(min_val+max_val)*2/3, max_val], "color": "#4b5563"},
            ],
        }
    ))
    fig.update_layout(height=250, margin=dict(l=10, r=10, t=40, b=10))
    return fig

def readability_page():
    if not st.session_state['user']: switch_page('login'); return

    st.title("📖 Text Readability Analyzer")

    # Input Method: Text or File
    tab1, tab2 = st.tabs(["✍️ Input Text", "📂 Upload File (TXT/PDF)"])
    text_input = ""

    with tab1:
        raw_text = st.text_area("Enter text to analyze (min 50 chars):", height=200)
        if raw_text: text_input = raw_text

    with tab2:
        uploaded_file = st.file_uploader("Upload a file", type=["txt", "pdf"])
        if uploaded_file:
            try:
                if uploaded_file.type == "application/pdf":
                    reader = PyPDF2.PdfReader(uploaded_file)
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text() + "\n"
                    text_input = text
                    st.info(f"✅ Loaded {len(reader.pages)} pages from PDF.")
                else:
                    text_input = uploaded_file.read().decode("utf-8")
                    st.info(f"✅ Loaded TXT file: {uploaded_file.name}")
            except Exception as e:
                st.error(f"Error reading file: {e}")

    if st.button("Analyze Readability", type="primary"):
        if len(text_input) < 50:
            st.error("Text is too short (min 50 chars). Please enter more text or upload a valid file.")
        else:
            import readability
            with st.spinner("Calculating advanced metrics..."):
                analyzer = readability.ReadabilityAnalyzer(text_input)
                score = analyzer.get_all_metrics()

            # --- Results Dashboard ---
            st.markdown("---")
            st.subheader("📊 Analysis Results")

            # 1. Overall Grade (Average)
            avg_grade = (score['Flesch-Kincaid Grade'] + score['Gunning Fog'] + score['SMOG Index'] + score['Coleman-Liau']) / 4

            # Determine Level
            if avg_grade <= 6: level, color = "Beginner (Elementary)", "#28a745"
            elif avg_grade <= 10: level, color = "Intermediate (Middle School)", "#17a2b8"
            elif avg_grade <= 14: level, color = "Advanced (High School/College)", "#ffc107"
            else: level, color = "Expert (Professional/Academic)", "#dc3545"

            st.markdown(f"""
            <div style="background-color: #1f2937; padding: 20px; border-radius: 10px; border-left: 5px solid {color}; text-align: center;">
                <h2 style="margin:0; color: {color} !important;">Overall Level: {level}</h2>
                <p style="margin:5px 0 0 0; color: #9ca3af;">Approximate Grade Level: {int(avg_grade)}</p>
            </div>
            """, unsafe_allow_html=True)

            st.markdown("### 📈 Detailed Metrics")

            # 2. Visual Gauges
            c1, c2, c3 = st.columns(3)
            with c1:
                st.plotly_chart(create_gauge(score["Flesch Reading Ease"], "Flesch Reading Ease", 0, 100, "#00ffcc"), use_container_width=True)
                with st.expander("ℹ️ About Flesch Ease"):
                    st.caption("0-100 Scale. Higher is easier. 60-70 is standard.")

            with c2:
                st.plotly_chart(create_gauge(score["Flesch-Kincaid Grade"], "Flesch-Kincaid Grade", 0, 20, "#ff00ff"), use_container_width=True)
                with st.expander("ℹ️ About Kincaid Grade"):
                    st.caption("US Grade Level. 8.0 means 8th grader can understand.")

            with c3:
                st.plotly_chart(create_gauge(score["SMOG Index"], "SMOG Index", 0, 20, "#ffff00"), use_container_width=True)
                with st.expander("ℹ️ About SMOG"):
                    st.caption("Commonly used for medical writing. Based on polysyllables.")

            c4, c5 = st.columns(2)
            with c4:
                st.plotly_chart(create_gauge(score["Gunning Fog"], "Gunning Fog", 0, 20, "#00ccff"), use_container_width=True)
                with st.expander("ℹ️ About Gunning Fog"):
                    st.caption("Based on sentence length and complex words.")

            with c5:
                st.plotly_chart(create_gauge(score["Coleman-Liau"], "Coleman-Liau", 0, 20, "#ff9900"), use_container_width=True)
                with st.expander("ℹ️ About Coleman-Liau"):
                    st.caption("Based on characters instead of syllables. Good for automated analysis.")

            # 3. Text Stats
            st.markdown("### 📝 Text Statistics")
            s1, s2, s3, s4, s5 = st.columns(5)
            s1.metric("Sentences", analyzer.num_sentences)
            s2.metric("Words", analyzer.num_words)
            s3.metric("Syllables", analyzer.num_syllables)
            s4.metric("Complex Words", analyzer.complex_words)
            s5.metric("Characters", analyzer.char_count)

def admin_page():
    st.title("👥 User Management")
    users = db.get_all_users()

    for u_email, u_created in users:
        c1, c2, c3 = st.columns([3, 2, 1])
        c1.write(f"**{u_email}**")
        c2.write(u_created)
        if u_email != "admin@llm.com":
            if c3.button("Delete", key=u_email):
                db.delete_user(u_email)
                st.warning(f"Deleted {u_email}")
                time.sleep(0.5)
                st.rerun()

# ---------------- FORGOT PASSWORD ----------------
def forgot_password():
      # Initialize session state (FIX)
    if "stage" not in st.session_state:
      st.session_state.stage = "email"

    if "reset_email" not in st.session_state:
        st.session_state.reset_email = ""

    if "otp_token" not in st.session_state:
        st.session_state.otp_token = None

    if "otp_sent_time" not in st.session_state:
        st.session_state.otp_sent_time = None

    st.markdown("""
    <div class="neon-card" style="
        display:flex;
        justify-content:center;
        align-items:center;
        text-align:center;
        font-size:28px;
        font-weight:600;
        letter-spacing:0.5px;
        color:#ffffff;
        text-shadow:0 0 12px rgba(0,245,255,0.8);
    ">
        Welcome to Infosys Springboard Internship
    </div>
    """, unsafe_allow_html=True)
    st.title("🔒 Forgot Password")

    # -------- STAGE 1: EMAIL --------
    if st.session_state.stage == "email":
        email = st.text_input("Enter your registered Email")

        if st.button("Verify Email"):
            if db.check_user_exists(email):
                st.session_state.reset_email = email
                st.session_state.email_verified = True
                st.success("Email Verified ✅")
            else:
                st.error("Email not found")

        # Show methods on SAME PAGE after verification
        if st.session_state.get("email_verified"):

            st.markdown("---")
            st.subheader("Choose Reset Method")

            col1, space, col2 = st.columns([1.3, 1, 1.3])

            with col1:
                if st.button("Reset via OTP", use_container_width=True):
                    st.session_state.stage = "otp"
                    st.rerun()

            with col2:
                if st.button("Reset via Security Question", use_container_width=True):
                    st.session_state.stage = "security"
                    st.rerun()
        st.markdown("<br><br>", unsafe_allow_html=True)
    # -------- STAGE 3A: OTP --------
    elif st.session_state.stage == "otp":
        st.subheader("OTP Verification")
        st.info(f"OTP will be sent to {st.session_state.reset_email}")

        OTP_VALID_SECONDS = 600

        # ---------- SEND OTP ----------
        if not st.session_state.otp_sent_time:
            if st.button("Send OTP"):
                otp = generate_otp()
                ok, msg = send_email(
                    st.session_state.reset_email,
                    otp,
                    EMAIL_PASSWORD
                )

                if ok:
                    st.session_state.otp_token = create_otp_token(
                        otp,
                        st.session_state.reset_email
                    )
                    st.session_state.otp_sent_time = time.time()
                    st.success("OTP sent successfully 📧")
                else:
                    st.error(f"Failed to send OTP: {msg}")

        # ---------- AFTER OTP SENT ----------
        if st.session_state.otp_sent_time:
            elapsed = time.time() - st.session_state.otp_sent_time
            remaining = int(OTP_VALID_SECONDS - elapsed)

            if remaining <= 0:
                st.error("OTP expired. Please resend.")
                st.session_state.otp_sent_time = None
                st.session_state.otp_token = None
            else:
                st.info(f"OTP expires in {remaining} seconds")

                otp_input = st.text_input("Enter OTP")

                col1, space, col2 = st.columns([1,2,1])

                # Verify
                if col1.button("Verify OTP",use_container_width=True):
                    if not otp_input.strip():
                        st.error("Please enter OTP")
                    else:
                        ok, msg = verify_otp_token(
                            st.session_state.otp_token,
                            otp_input.strip(),
                            st.session_state.reset_email
                        )
                        if ok:
                            st.success("OTP Verified ✅")
                            st.session_state.stage = "reset"
                            st.session_state.otp_sent_time = None
                            st.rerun()
                        else:
                            st.error("Invalid OTP")

                # Resend
                if col2.button("Resend OTP",use_container_width=True):
                    st.session_state.otp_sent_time = None
                    st.session_state.otp_token = None
                    st.rerun()

    # -------- STAGE 3B: SECURITY QUESTION --------
    elif st.session_state.stage == "security":
        question = db.get_security_question(st.session_state.reset_email)
        st.write(f"Security Question: {question}")

        answer = st.text_input("Enter Answer")

        if st.button("Verify Answer"):
            if db.verify_security_answer(
                st.session_state.reset_email,
                answer
            ):
                st.session_state.stage = "reset"
                st.success("Answer Verified ✅")
                st.rerun()
            else:
                st.error("Incorrect Answer")

    # -------- STAGE 4: RESET PASSWORD --------
    elif st.session_state.stage == "reset":
        new_pass = st.text_input("New Password", type="password")
        confirm_pass = st.text_input("Confirm Password", type="password")

        if st.button("Update Password"):
            if not valid_password(new_pass):
                st.error("Weak password")
            elif new_pass != confirm_pass:
                st.error("Passwords do not match")
            elif db.check_password_reused(
                st.session_state.reset_email,
                new_pass
            ):
                st.error("You cannot reuse old password")
            else:
                db.update_password(
                    st.session_state.reset_email,
                    new_pass
                )
                st.success("Password Updated Successfully 🎉")

    if st.button("Back to Login"):
        st.session_state.stage = "email"
        st.session_state.page = "login"
        st.rerun()


    st.markdown('</div>', unsafe_allow_html=True)

# ---------------- ROUTING ----------------
# Default page

# Initialize session values
if "user" not in st.session_state:
    st.session_state.user = None

if "page" not in st.session_state:
    st.session_state.page = "login"


# ------------------ MAIN ROUTER ------------------

if st.session_state.user:
    # -------- Logged-in Layout --------
    with st.sidebar:
        st.image(
            "https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Infosys_logo.svg/1280px-Infosys_logo.svg.png",
            width=150
        )

        try:
          payload = jwt.decode(
              st.session_state.token,
              SECRET_KEY,
              algorithms=[ALGORITHM]
          )
          email = payload.get("email")
          username = payload.get("username", email)
        except:
          st.session_state.token = None
          st.session_state.page = "login"
          st.rerun()

        # Sidebar
        st.sidebar.markdown(f"""
        <div style="font-size:20px; font-weight:bold; margin-bottom:10px;">
        👤 {username}
        </div>
        """, unsafe_allow_html=True)
        st.markdown("---")

        opts = ["Chat", "Readability"]
        icons = ["chat-dots", "book"]

        # Admin option
        if st.session_state.user == "admin@llm.com":
            opts.append("Admin")
            icons.append("shield-lock")

        selected = option_menu(
            "Infosys Springboard",
            opts,
            icons=icons,
            menu_icon="cast",
            default_index=0,
            styles={
                "container": {
                    "background-color": "#0e1a24",
                    "padding": "10px"
                },
                "icon": {
                    "color": "#00F5FF;",
                    "font-size": "18px"
                },
                "nav-link": {
                    "color": "#cfd8dc",
                    "font-size": "16px",
                    "text-align": "left",
                    "margin": "5px",
                    "--hover-color": "#123344",
                },
                "nav-link-selected": {
                    "background-color": "#123344",
                    "color": "#00F5FF;",
                    "border-radius": "10px",
                    "box-shadow": "0 0 10px #00F5FF;"
                },
            }
        )

        st.markdown("---")
        if st.button("🔓 Log Out"):
            st.session_state.clear()
            st.session_state.page = "login"
            st.rerun()

    # -------- Page Selection --------
    if selected == "Chat":
        dashboard()

    elif selected == "Readability":
        readability_page()

    elif selected == "Admin":
        admin_page()

else:
    # -------- Not Logged In --------
    if st.session_state.page == "login":
        login()

    elif st.session_state.page == "signup":
        signup()

    elif st.session_state.page == "forgot":
        forgot_password()


Overwriting app.py


In [ ]:
import os
import subprocess
import time
from google.colab import userdata
from pyngrok import ngrok

# 1. Retrieve secrets safely in the kernel
email_pass = None
ngrok_token = None

try:
    try:
        email_pass = userdata.get('EMAIL_PASSWORD')
    except Exception as e:
        print(f"⚠️ Warning: EMAIL_PASSWORD secret not found: {e}")

    try:
        ngrok_token = userdata.get('NGROK_AUTHTOKEN')
    except Exception as e:
        print(f"⚠️ Warning: NGROK_AUTHTOKEN secret not found: {e}")

    # Store in environment for the subprocess to see
    if email_pass:
        os.environ['EMAIL_PASSWORD'] = email_pass
    os.environ['JWT_SECRET'] = "super-secret-change-me"

except Exception as e:
    print(f"❌ Error setting up environment: {e}")

# 2. Authenticate Ngrok
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)

    # 3. Kill old processes to prevent conflicts
    ngrok.kill()
    time.sleep(1)

    # 4. Run Streamlit in the background
    # We pass os.environ so it inherits the secrets we just set
    process = subprocess.Popen(['streamlit', 'run', 'app.py'], env=os.environ.copy())

    # Give it a moment to start
    time.sleep(3)

    # 5. Open Tunnel
    try:
        public_url = ngrok.connect(8501).public_url
        print(f"\n🚀 Your App is running at: {public_url}")
        print("\n👇 Click the link above to open the app!")
    except Exception as e:
        print(f"❌ Error starting Ngrok tunnel: {e}")

    # 6. Keep Alive / Manual Stop
    print("\n✅ App is running! Check the URL above.")
    try:
        input("\n🛑 Press ENTER in this box to STOP the server...\n")
    except (KeyboardInterrupt, EOFError):
        pass
    finally:
        process.terminate()
        ngrok.kill()
        print("✅ Server and Tunnel stopped.")

else:
    print("❌ No Ngrok Token found. Please add 'NGROK_AUTHTOKEN' to Colab Secrets.")



🚀 Your App is running at: https://postrorse-unstatesmanlike-deloras.ngrok-free.dev

👇 Click the link above to open the app!

✅ App is running! Check the URL above.
